In [110]:
# <exp> ::= 0 | 1 | 2
# <exp> ::= '👍' | '👎'
# <exp> ::= (<Operator> <exp> <exp>)
# <exp> ::= ('|' <exp> <exp) | ('&' <exp> <exp>) | ('❗' <exp>)
# <numericalOperator> ::= '+' | '-' | '*' | '/'
# <booleanOperator> ::= '|' | '&' | '❗' 

In [111]:
import random

def tokenize(string):
  #inputs a string containing a Delta expression
  #returns a list of atoms of the language
  return string.replace('(', ' ( ').replace(')', ' )').split()

def parse(tokenList):
  #inputs a list of tokens
  #returns a parse tree, represented as a nested list structure
  token = tokenList.pop(0)
  if token[0].isdigit():
    return int(token)
  # just a terminal
  if token[0] != '(':
    return token
  #  <exp> can be (<operator> <exp>) or (<operator> <exp> <exp>) or (<operator> followed by any number of expressions
  ans = [tokenList.pop(0)]
  while tokenList[0] != ')':
    ans.append(parse(tokenList)) 
  tokenList.pop(0) # to get rid of the ')'
  return ans

# print(parse(tokenize(deltaCode)))

#check that the parentheses balance
def balanced(tokenList):
  count = 0
  for token in tokenList:
    if token == '(':
      count+= 1
    if token == ')':
      count-= 1
    if count < 0:
      return False
  return count == 0

# print(balanced(tokenize(deltaCode)))

def prettyPrint(exp, depth = 0):
  if isinstance(exp,int):
    print(" "*3*depth + str(exp))
  else:
    print(" "*3*depth + "(" + str(exp[0]))
    prettyPrint(exp[1], depth+1)
    prettyPrint(exp[2], depth+1)
    print(" "*3*depth + ")")

# prettyPrint(parse(tokenize(deltaCode)))

def run(program):
  # input string, output the execution of the Delta program
  return evalDelta(parse(tokenize(program)))

def evalDelta(exp):
  if isinstance(exp, int):
    return exp
  if exp == '👍':
    return True
  if exp == '👎':
    return False
  if exp[0] == '❗':
    return not evalDelta(exp[1])
  if exp[0] == '|':
    return evalDelta(exp[1]) or evalDelta(exp[2])
  if exp[0] == '&':
    return evalDelta(exp[1]) and evalDelta(exp[2])
  #know [<operator> <exp> <exp>]
  if exp[0] == '+':
    return evalDelta(exp[1]) + evalDelta(exp[2])
  if exp[0] == '-':
    return evalDelta(exp[1]) - evalDelta(exp[2])
  if exp[0] == '/':
    return evalDelta(exp[1]) // evalDelta(exp[2])
  if exp[0] == '*':
    return evalDelta(exp[1]) * evalDelta(exp[2])
  if exp[0] == '==':
    return '👍' if evalDelta(exp[1]) == evalDelta(exp[2]) else '👎'
  if exp[0] == '<':
    return '👍' if evalDelta(exp[1]) < evalDelta(exp[2]) else '👎'
  if exp[0] == 'if':
    # try block for catching nested conditionals
    if evalDelta(exp[1]):
      try:
        return evalDelta(exp[2])
      except:
        return '👍'
    else:
      try:
        return evalDelta(exp[3])
      except:
        return '👎'

# random program genereator
def randPG(programType):
  if programType == "boolean":
    return generateBoolean()
  elif programType == "numerical":
    return "(" + generateOperator("numerical") + " " + generateExp("numerical") + " " + generateExp("numerical") + ")"
  elif programType == "conditional":
    return "(if " + randPG("boolean") + " " + randPG("numerical") + " " + randPG("numerical") + ") "
  else:
    return "ERROR: INVALID PROGRAM TYPE REQUESTED"

def generateBoolean():
  operator = generateOperator("boolean")
  output = "(" + operator + " "
  if operator == "<":
    output += generateExp("numerical") + " " + generateExp("numerical")
  else:
    output += generateExp("boolean")
    if operator != "❗":
      output +=  " " + generateExp("boolean")
  return output + ")"

def generateExp(expType):
  if random.choice([True, False, False, False]): # arbitrary list, determines how often exp are recursive
    if expType == "numerical":
      return randPG("numerical")
    elif expType == "boolean":
      return randPG("boolean")
  else:
    if expType == "numerical":
      return str(random.randint(0, 1000)) # Arbitrary numbers, chose small numbers to increase readability
    elif expType == "boolean":
      return '👍' if random.choice([True, False]) else '👎'

def generateOperator(expType):
  # TERNARY: return random.choice(["+", "-", "*", "/"]) if type == "numerical" else random.choice(["==", "<", "|", "&", "❗"])
  if expType == "numerical":
    return random.choice(["+", "-", "*", "/"]) 
  elif expType == "boolean":
    return random.choice(["==", "<", "|", "&", "❗"])


In [112]:
print("examples of '==' '<' and 'if")
print("(== 👍 1):\t" + run("(== 👍 1)"))
print("(== 0 0):\t" + run("(== 0 0)"))
print("(< 79 80):\t" + run("(< 79 80)"))
print("(< 81 80):\t" + run("(< 81 80)"))
print("(< 80 80):\t" + run("(< 80 80)"))
print("(if (< 81 80) 4 5):\t" + str(run("(if (< 81 80) 4 5)")))
print("(if (== 80 80) 4 5):\t" + str(run("(if (== 80 80) 4 5)")))

print("\nexamples of randPG")
print(randPG("numerical"))
print(randPG("boolean"))
print(randPG("conditional"))

examples of '==' '<' and 'if
(== 👍 1):	👍
(== 0 0):	👍
(< 79 80):	👍
(< 81 80):	👎
(< 80 80):	👎
(if (< 81 80) 4 5):	4
(if (== 80 80) 4 5):	4

examples of randPG
(/ (+ 8 477) (* (+ 139 945) 253))
(< (+ 868 (- 812 715)) 203)
(if (& (== 👍 (❗ (& (< 623 (+ 775 98)) 👍))) 👎) (- 470 498) (/ 618 (+ 124 (+ 181 532)))) 
